## Анализ Рыночных Цен Автомобилей с Построчной Обработкой и Анализом в Hive

__Цель:__

Создать программу на Python для анализа средних рыночных цен на автомобили различных марок с последующей обработкой данных в Apache Hive для определения самой дорогой и самой дешевой марки автомобиля. 

__Датасет:__ 

Используйте доступный на Kaggle датасет, включающий марку, модель, цену и другие характеристики автомобилей [](https://www.kaggle.com/datasets/CooperUnion/cardataset) .

__Основные Задачи:__

Загрузка Данных: Автоматическая загрузка датасета, если он отсутствует

__Построчная Обработка Данных:__

Используйте map и reduce для обработки данных из CSV-файла построчно.

mapper: извлекает марку и цену из каждой строки.

reducer: накапливает данные по маркам и ценам.

__Анализ Данных:__

Вычисление средней цены для каждой марки.

__Сохранение Результатов:__

Сохраните результаты в новый CSV-файл.

In [2]:
import pandas as pd

cars = pd.read_csv("data.csv", engine='python', encoding='utf-8', on_bad_lines="skip", encoding_errors="ignore")
cars.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [4]:
# убираем ненужные данные и сохраняем:
car_prices = pd.concat([cars['Make'], cars['MSRP']], axis=1) 
car_prices = car_prices.sort_values(by=['Make'])
car_prices.to_csv('car_prices.csv', header=False)
car_prices = [tuple(row) for row in car_prices.to_numpy()]
#print(car_prices)


In [5]:
from functools import reduce

def chunkify(car_price_list):
    # разбивает список моделей по маркам
    i = 0
    j = 0
    while i < len(car_price_list):
        while j < len(car_price_list) and car_price_list[i][0] == car_price_list[j][0]:
            j += 1
        yield car_price_list[i : j]
        i = j
    

def mapper(car_models):
    # находит среднюю цену для марки автомобиля
    summ = 0
    for m in car_models:
        summ += m[1]
    mean = summ / len(car_models)
    return (car_models[0][0], mean, )

def reducer(price_data1, price_data2):
        # находит самую дешевую и самую дорогую марку автомобиля
        if isinstance(price_data1[1], float):
            # на первом шаге
            pmin = price_data1 if price_data1[1] < price_data2[1] else price_data2
            pmax = price_data1 if price_data1[1] > price_data2[1] else price_data2
        else:
            # на следующих шагах
            pmin = price_data1[0] if price_data1[0][1] < price_data2[1] else price_data2
            pmax = price_data1[1] if price_data1[1][1] > price_data2[1] else price_data2
        return pmin, pmax 
    
data_chunks = chunkify(car_prices)
mapped = map(mapper, data_chunks)
(pmin, pmax) = reduce(reducer, mapped)
print("Самая дешевая марка: ", pmin)
print("Самая дорогая марка: ", pmax)

Самая дешевая марка:  ('Plymouth', 3122.9024390243903)
Самая дорогая марка:  ('Bugatti', 1757223.6666666667)
